# Pump it Up: Data Mining the Water Table

## Data load and preprocessing

### Data load

In [2]:
# DATA
# =======================
import pandas as pd
import numpy as np
import time
pd.set_option('display.max_columns', 0)

#Visualization
# =======================
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt

In [3]:
X_train = pd.read_csv('../data/train.csv')
X_test = pd.read_csv('../data/test.csv')

y_train = pd.read_csv('../data/target_train.csv')
y_test = pd.DataFrame([X_test['id'].values,['0,' * (len(X_test)-1)][0].split(',')])
y_test = y_test.transpose()
y_test.columns = ['id','status_group']

In [4]:
X_train = X_train.drop_duplicates()

In [5]:
X_train = X_train.merge(y_train, on='id')
X_test = X_test.merge(y_test, on='id')

In [6]:
df = pd.concat([X_train,X_test])

### Preprocessing

#### Nulls

In [7]:
from sklearn.impute import MissingIndicator
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [8]:
print("Nulls:")
cols_nul= list()
for col in df.columns :
        if df[col].isna().sum() *100 / len(df) >0.0:
            cols_nul += [col]
            print(f"\t{col}: {round(df[col].isna().sum() *100 / len(df),3)}")

Nulls:
	funder: 6.066
	installer: 6.104
	subvillage: 0.633
	public_meeting: 5.596
	scheme_management: 6.527
	scheme_name: 47.486
	permit: 5.108


Scheme_name tiene un porcentaje de null muy alto, el resto está cerca del 5% por lo que podemos imputar estos

In [9]:
simp_imp = SimpleImputer(strategy=  'most_frequent')
miss_ind = MissingIndicator()

In [10]:
df['funder'] = simp_imp.fit_transform(df[['funder']])
df['subvillage'] = simp_imp.fit_transform(df[['subvillage']])
df['installer'] = simp_imp.fit_transform(df[['installer']])
df['public_meeting'] = simp_imp.fit_transform(df[['public_meeting']])
df['scheme_management'] = simp_imp.fit_transform(df[['scheme_management']])
df['scheme_name'] = miss_ind.fit_transform(df[['scheme_name']])
df['permit'] = simp_imp.fit_transform(df[['permit']])

#### Analisis por tipo de columna

##### Date

In [11]:
df['year'] = df['date_recorded'].apply(lambda x : int(str(x).split('-')[0]))
df['month'] = df['date_recorded'].apply(lambda x : int(str(x).split('-')[1]))
df['day'] = df['date_recorded'].apply(lambda x : int(str(x).split('-')[1]))

#for impute 0s
most_freq_cons = {
    2001 : df[['year','construction_year']].value_counts().loc[2001].index[0] ,
    2002 : df[['year','construction_year']].value_counts().loc[2001].index[0] ,
    2004 : df[['year','construction_year']].value_counts().loc[2004].index[1] ,
    2011 : df[['year','construction_year']].value_counts().loc[2011].index[1] ,
    2012 : df[['year','construction_year']].value_counts().loc[2012].index[1] ,
    2013 : df[['year','construction_year']].value_counts().loc[2013].index[1] }
ind_cons_cero = df[df['construction_year'] == 0].index
df.loc[ind_cons_cero, 'construction_year'] = df.loc[ind_cons_cero,'year'].apply(lambda x: most_freq_cons[x] )

df['dif_year'] = df['year'] - df['construction_year']

seas = {
    1 : 0,
    2 : 0,
    3 : 1,
    4 : 1,
    5 : 1,
    6 : 2,
    7 : 2,
    8 : 2,
    9 : 3,
    10: 3,
    11: 3,
    12: 0,
}
df['seas'] = df['month'].apply(lambda x : seas[x])

##### Categorical

In [12]:
df['public_meeting'] = df['public_meeting'].apply(lambda x: 0 if x == False else 1)
df['scheme_name'] = df['scheme_name'].apply(lambda x: 0 if x == False else 1)
df['permit'] = df['permit'].apply(lambda x: 0 if x == False else 1)


df['waterpoint_type_group'] = df['waterpoint_type_group'].apply(lambda x : x if x in set(['communal standpipe','hand pump']) else 'other'  )
df['source_class'] = df['source_class'].apply(lambda x : x  if x == 'groundwater' else 'surface')
df['source_type'] = df['source_type'].apply(lambda x : x if x in set(['spring','shallow well','borehole']) else 'other' )
df['quantity_group'] = df['quantity_group'].apply(lambda x : x if x in set(['enough','insufficient','dry']) else 'dry' )
df['water_quality'] = df['water_quality'].apply(lambda x : x if x == 'soft' else 'other')
df['payment'] = df['payment'].apply(lambda x : 'pay' if x in set(['pay per bucket','pay monthly','pay when scheme fails','pay annually']) else x if x == 'noPay' else 'other' )
df['management'] = df['management_group'].apply(lambda x : 'private' if x == 'user-group' else 'other')
df['extraction_type'] = df['extraction_type_class'].apply(lambda x : x if x in set(['gravity','handpump']) else 'other' )

df['funder'] = df['funder'].apply(lambda x : x.lower())
df['installer'] = df['installer'].apply(lambda x : x.lower())
df['fun_ins'] = df['funder'] == df['installer'] 
df['fun_ins'] = df['fun_ins'].apply(lambda x: 1 if x == True else 0 )
df['fun_ins2'] = df['funder'].map(lambda x: x[:2]) == df['installer'].map(lambda x: x[:2]) 
df['fun_ins2'] = df['fun_ins2'].apply(lambda x: 1 if x == True else 0 )
df['fun_ins'] = (df['fun_ins']+df['fun_ins2'])>0
df['fun_ins'] = df['fun_ins'].apply(lambda x: 1 if x == True else 0 )
df['installer'] = df['installer'].apply(lambda x : x if x=='dwe' else 'other')

df['basin'] = df['basin'].apply(lambda x : x[:4] if x[:4] == 'Lake' else 'other')

df['wpt_name'] = df['wpt_name'].apply(lambda x : x[:3])
df['subvillage'] = df['subvillage'].apply(lambda x : x[:3])
df['fun_wpt'] = df['wpt_name'] == df['subvillage'] 
df['fun_wpt'] = df['fun_wpt'].apply(lambda x: 1 if x == True else 0 )


df['lga'] = df['lga'].apply(lambda x : x[:3])
df['ward'] = df['ward'].apply(lambda x : x[:3])
df['fun_lga'] = df['lga'] == df['ward'] 
df['fun_lga'] = df['fun_lga'].apply(lambda x: 1 if x == True else 0 )

df = df.drop(['waterpoint_type','source','quantity','quality_group','payment_type','management_group','extraction_type_group','extraction_type_class','recorded_by','scheme_management','fun_ins2','funder',
           'num_private','wpt_name','subvillage','lga','ward','fun_lga'], axis =1)

##### Numerical

In [60]:
import math
from sklearn.preprocessing import MinMaxScaler, RobustScaler

In [94]:
df['radio'] = df['gps_height']**2 +df['longitude']**2 + df['latitude']**2
df['radio'] = df.radio.apply(lambda x: math.sqrt(x))
df['vol'] = df.radio.apply(lambda x : (math.pi*4*(x**3))/3 )
esc = MinMaxScaler()
df[['radio','vol']] = esc.fit_transform(df[['radio','vol']])

esc = RobustScaler()
df[['amount_tsh','population']] = esc.fit_transform(df[['amount_tsh','population']])
df = df.drop(['gps_height','longitude','latitude'], axis =1)

#### Analisis de variables

In [ ]:
df = df.drop(['date_recorded', 'day','construction_year'], axis = 1)

In [107]:
X_train = df[:len(X_train)]
X_test = df[len(X_train):]

y_train = X_train[['id','status_group']]
y_test = X_test[['id','status_group']]

X_train = X_train.drop(['status_group'], axis = 1)
X_test = X_test.drop(['status_group'], axis = 1)

In [111]:
numerical = ['amount_tsh', 'population', 'radio', 'vol' ]
categorical = ['installer', 'basin', 'region',
       'region_code', 'district_code', 'public_meeting',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'management', 'payment', 'water_quality', 'quantity_group',
       'source_type', 'source_class', 'waterpoint_type_group', 'year', 'month',
        'dif_year', 'seas', 'fun_ins', 'fun_wpt']

In [113]:
df[categorical]

,installer,basin,region,region_code,district_code,public_meeting,scheme_name,permit,construction_year,extraction_type,management,payment,water_quality,quantity_group,source_type,source_class,waterpoint_type_group,year,month,dif_year,seas,fun_ins,fun_wpt
0,other,Lake,Iringa,11,5,1,0,0,1999,gravity,private,pay,soft,enough,spring,groundwater,communal standpipe,2011,3,12,1,1,0
1,other,Lake,Mara,20,2,1,1,1,2010,gravity,private,other,soft,insufficient,other,surface,communal standpipe,2013,3,3,1,1,0
2,other,other,Manyara,21,4,1,0,1,2009,gravity,private,pay,soft,enough,other,surface,communal standpipe,2013,2,4,0,0,0
3,other,other,Mtwara,90,63,1,1,1,1986,other,private,other,soft,dry,borehole,groundwater,communal standpipe,2013,1,27,0,1,0
4,other,Lake,Kagera,18,1,1,1,1,2009,gravity,other,other,soft,dry,other,surface,communal standpipe,2011,7,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,other,other,Pwani,6,1,1,0,1,2009,other,private,other,soft,enough,other,surface,communal standpipe,2011,2,2,0,1,0
14846,other,other,Tanga,4,7,1,1,0,2009,handpump,private,pay,other,insufficient,shallow well,groundwater,hand pump,2011,3,2,1,1,0
14847,dwe,other,Singida,13,2,1,1,1,2010,gravity,private,other,soft,insufficient,other,surface,communal standpipe,2013,3,3,1,0,0
14848,dwe,Lake,Ruvuma,10,2,1,0,1,2009,gravity,private,other,soft,insufficient,other,surface,communal standpipe,2013,2,4,0,0,0
